<a href="https://colab.research.google.com/github/tug98850/Lyft_Data_Challenge/blob/master/2019_Lyft_Data_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from collections import Counter

#### Want:

price per ride *for each driver* 

-- each driver will have multiple rides
--> want an average for each driver

In [0]:
driver_link = 'https://raw.githubusercontent.com/tug98850/Lyft_Data_Challenge/master/driver_ids.csv'
ts_link = 'https://raw.githubusercontent.com/tug98850/Lyft_Data_Challenge/master/ride_timestamps.csv'
ride_link = 'https://raw.githubusercontent.com/tug98850/Lyft_Data_Challenge/master/ride_ids.csv'

d = pd.read_csv(driver_link)
r = pd.read_csv(ride_link)
ts = pd.read_csv(ts_link)

In [0]:
d.head(2)

,driver_id,driver_onboard_date
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29 00:00:00
1,007f0389f9c7b03ef97098422f902e62,2016-03-29 00:00:00


In [0]:
r.head(2)

,driver_id,ride_id,ride_distance,ride_duration,ride_prime_time
0,002be0ffdc997bd5c50703158b7c2491,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50
1,002be0ffdc997bd5c50703158b7c2491,01b522c5c3a756fbdb12e95e87507eda,3362,809,0


In [0]:
ts.head(5)

,ride_id,event,timestamp
0,00003037a262d9ee40e61b5c0718f7f0,requested_at,2016-06-13 09:39:19
1,00003037a262d9ee40e61b5c0718f7f0,accepted_at,2016-06-13 09:39:51
2,00003037a262d9ee40e61b5c0718f7f0,arrived_at,2016-06-13 09:44:31
3,00003037a262d9ee40e61b5c0718f7f0,picked_up_at,2016-06-13 09:44:33
4,00003037a262d9ee40e61b5c0718f7f0,dropped_off_at,2016-06-13 10:03:05


In [0]:
d.describe()

,driver_id,driver_onboard_date
count,937,937
unique,937,49
top,b5d89a7f152103c9fc981e3990e9fc86,2016-04-05 00:00:00
freq,1,36


In [0]:
r.describe()

,ride_distance,ride_duration,ride_prime_time
count,193502.000000,193502.000000,193502.000000
mean,6955.218266,858.966099,17.305893
std,8929.444606,571.375818,30.825800
min,-2.000000,2.000000,0.000000
25%,2459.000000,491.000000,0.000000
50%,4015.000000,727.000000,0.000000
75%,7193.000000,1069.000000,25.000000
max,724679.000000,28204.000000,500.000000


In [0]:
ts.describe()

,ride_id,event,timestamp
count,970405,970405,970404
unique,194081,5,865826
top,43a0f81774dd8e6e3cf8473287a4e086,accepted_at,2016-04-30 22:09:15
freq,5,194081,7


**General statistics**

There are 937 different drivers.

The mean ride distance is 6955.21 meters.

The mean ride duration is 14.3 minutes.

The mean prime time is 17 seconds.

In [0]:
# calculate each driver's life time
# life_time = last_time_ride - onboard_time
# also check for if a user onboard but have not given a ride

# check driver_onboard_date col
print(d['driver_onboard_date'].dtype)

#check timestamp
print(ts['timestamp'].dtype)

object
object


In [0]:
d['driver_onboard_date'] = pd.to_datetime(d['driver_onboard_date'])
ts['timestamp'] = pd.to_datetime(ts['timestamp'])

In [0]:
# merging driver and ride first
merged = pd.merge(d, r, on = "driver_id")
# merging d, r and timestamp
merged = pd.merge(merged, ts, on = "ride_id")

merged['last_time_drove'] = merged.groupby(['driver_id'])['timestamp'].transform(max)

# for each driver, last_time_drove - on_board_date
merged['life_time'] = merged['last_time_drove'] - merged['driver_onboard_date']
merged.head(5)

,driver_id,driver_onboard_date,ride_id,ride_distance,ride_duration,ride_prime_time,event,timestamp,last_time_drove,life_time
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,requested_at,2016-04-23 02:13:50,2016-06-23 10:29:53,86 days 10:29:53
1,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,accepted_at,2016-04-23 02:14:15,2016-06-23 10:29:53,86 days 10:29:53
2,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,arrived_at,2016-04-23 02:16:36,2016-06-23 10:29:53,86 days 10:29:53
3,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,picked_up_at,2016-04-23 02:16:40,2016-06-23 10:29:53,86 days 10:29:53
4,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,dropped_off_at,2016-04-23 02:22:07,2016-06-23 10:29:53,86 days 10:29:53


In [0]:
# verifying whether we calculated last_time_drove
# by finding the max timestamp value of driver
# 002be0ffdc997bd5c50703158b7c2491
print('Last time drove is:')
print(max(merged[merged['driver_id'] == '002be0ffdc997bd5c50703158b7c2491']['timestamp'].unique()))
merged[merged['driver_id'] == '002be0ffdc997bd5c50703158b7c2491']['last_time_drove'].head(5)

Last time drove is:
2016-06-23T10:29:53.000000000


0   2016-06-23 10:29:53
1   2016-06-23 10:29:53
2   2016-06-23 10:29:53
3   2016-06-23 10:29:53
4   2016-06-23 10:29:53
Name: last_time_drove, dtype: datetime64[ns]

Verified that we calculated last time drove correctly.

In [0]:
# verifying if the last time drove we computed
# was reasonable
print('Checking last time drove')
print(merged['last_time_drove'].describe())

Checking last time drove
count                  921045
unique                    836
top       2016-06-26 19:04:27
freq                     4945
first     2016-03-30 17:45:36
last      2016-06-27 00:50:50
Name: last_time_drove, dtype: object


In [0]:
print('Checking life time value')
print(merged['life_time'].describe())

Checking life time value
count                     921045
mean     65 days 01:28:40.248473
std      16 days 10:54:33.353497
min              1 days 17:45:36
25%             53 days 14:40:53
50%             66 days 14:17:39
75%             78 days 21:51:06
max             90 days 18:55:28
Name: life_time, dtype: object


According to the life time we calculated, an average Lyft
driver drives for about 2 months. 

**Calculate life-time for each driver**

We just calculate the life time of each value and put it in the life_time column of the merged table

In [0]:
# calculate fare
# fare = base + costPerMile(totalMile) + costPerMinute(totalMinute)
# we are going to do this for every ride_id
costPerMile = 1.15
costPerMinute = 0.22

# convert since ride_distance is in meters and ride_duration is in seconds
costPerMeter = 1.15 * (1 / 1609.34)
costPerSecond = 0.22 * (1 / 60)
merged['fare'] = 2.00 + costPerMeter * merged['ride_distance'] + costPerSecond *  merged['ride_duration']
merged.head(10)

,driver_id,driver_onboard_date,ride_id,ride_distance,ride_duration,ride_prime_time,event,timestamp,last_time_drove,life_time,fare
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,requested_at,2016-04-23 02:13:50,2016-06-23 10:29:53,86 days 10:29:53,4.493102
1,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,accepted_at,2016-04-23 02:14:15,2016-06-23 10:29:53,86 days 10:29:53,4.493102
2,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,arrived_at,2016-04-23 02:16:36,2016-06-23 10:29:53,86 days 10:29:53,4.493102
3,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,picked_up_at,2016-04-23 02:16:40,2016-06-23 10:29:53,86 days 10:29:53,4.493102
4,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,dropped_off_at,2016-04-23 02:22:07,2016-06-23 10:29:53,86 days 10:29:53,4.493102
5,002be0ffdc997bd5c50703158b7c2491,2016-03-29,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,requested_at,2016-03-29 19:00:49,2016-06-23 10:29:53,86 days 10:29:53,7.368747
6,002be0ffdc997bd5c50703158b7c2491,2016-03-29,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,accepted_at,2016-03-29 19:00:52,2016-06-23 10:29:53,86 days 10:29:53,7.368747
7,002be0ffdc997bd5c50703158b7c2491,2016-03-29,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,arrived_at,2016-03-29 19:03:57,2016-06-23 10:29:53,86 days 10:29:53,7.368747
8,002be0ffdc997bd5c50703158b7c2491,2016-03-29,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,picked_up_at,2016-03-29 19:04:01,2016-06-23 10:29:53,86 days 10:29:53,7.368747
9,002be0ffdc997bd5c50703158b7c2491,2016-03-29,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,dropped_off_at,2016-03-29 19:17:30,2016-06-23 10:29:53,86 days 10:29:53,7.368747


**Fare for each ride**

Calculated each ride's fare based on this formula.
fare = base + costPerMile(totalMile) + costPerMinute(totalMinute)

In [0]:
# calculate total fare each driver generate
# this will create a new Data Frame merged.groupby(['driver_id'])['fare'].sum()

# this will make a new column in the existing dataframe transform(sum)
merged['driver_revenue'] = merged.groupby(['driver_id'])['fare'].transform(sum)

# divide by 5 because for each ride_id, we have 5 events and so if we sum
# the fare column up, we will add the same fare for a ride 5 times
merged['driver_revenue'] /= 5
merged.head(5)

# the reason why groupby driver_id, ride_id does not work well
# because it will actually aggregate the table like this
# driver_1 ride_1 <- if we use sum, it will find the sum of the fare for this instance (since each ride has 5 events)
#                    the result will just be the fare * 5 and then it will move on to do the exact same thing for ride_2
#          ride_2
#          ride_3

,driver_id,driver_onboard_date,ride_id,ride_distance,ride_duration,ride_prime_time,event,timestamp,last_time_drove,life_time,fare,driver_revenue
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,requested_at,2016-04-23 02:13:50,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793
1,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,accepted_at,2016-04-23 02:14:15,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793
2,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,arrived_at,2016-04-23 02:16:36,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793
3,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,picked_up_at,2016-04-23 02:16:40,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793
4,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,dropped_off_at,2016-04-23 02:22:07,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793


In [0]:
# verifying whether we calculated total revenue
# by summing all unique fare values of driver
# 002be0ffdc997bd5c50703158b7c2491
print('Revenue is:')
print(sum(merged[merged['driver_id'] == '002be0ffdc997bd5c50703158b7c2491']['fare'].unique()))
merged[merged['driver_id'] == '002be0ffdc997bd5c50703158b7c2491']['driver_revenue'].head(5)

Revenue is:
2608.777930107995


0    2608.77793
1    2608.77793
2    2608.77793
3    2608.77793
4    2608.77793
Name: driver_revenue, dtype: float64

We expected all the value in the driver_revenue column
to be the same and they are the same.

We used another way to calculate the sum. That is finding all unique fare value for a user and sum it up.

The check verifies that the driver_revenue is correct.

In [0]:
merged['driver_revenue'].describe()

count    921045.000000
mean       3651.984571
std        1732.330580
min          19.348089
25%        2496.077038
50%        3447.223276
75%        4659.086871
max        9121.902143
Name: driver_revenue, dtype: float64

**Calculate the total fare each driver generate**

I used .transform() to make sure it creates another Pandas Column in the existing DF.

In [0]:
# calculate life time value
# life_time_val = driver_revenue / life_time (for each driver)

# converting life_time to number of days in float
#merged['life_time_float'] = merged['life_time'].days
merged['life_time_revenue'] = merged['driver_revenue'] / (merged['life_time'] / pd.to_timedelta(1, unit='D'))
merged.head(5)

,driver_id,driver_onboard_date,ride_id,ride_distance,ride_duration,ride_prime_time,event,timestamp,last_time_drove,life_time,fare,driver_revenue,life_time_revenue
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,requested_at,2016-04-23 02:13:50,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793,30.181118
1,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,accepted_at,2016-04-23 02:14:15,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793,30.181118
2,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,arrived_at,2016-04-23 02:16:36,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793,30.181118
3,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,picked_up_at,2016-04-23 02:16:40,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793,30.181118
4,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,dropped_off_at,2016-04-23 02:22:07,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793,30.181118


**Calculate life_time_revenue**

This is equivalent to how much revenue a Lyft Driver makes in a day. 

In [0]:
# get summary statistic for life_time_revenue
merged.describe()['life_time_revenue']

count    921045.000000
mean         57.294050
std          27.452919
min           0.458202
25%          37.731548
50%          53.704554
75%          73.564973
max         201.326519
Name: life_time_revenue, dtype: float64

**Summary stat for life time revenue**
On average, a driver generates about $57 dollars per day for Lyft. (account for both work and non-work days as well)

The next question is on the days that they do work, how much do they generate for Lyft? And on those workday, how far and long do they have to drive?


In [0]:
# calculating revenue per ride
value_df = merged.groupby('driver_id')['driver_revenue'].unique().values / merged.groupby('driver_id')['ride_id'].nunique()
value_df = pd.DataFrame({'driver_id': value_df.index, 'avg_rev_per_ride' : value_df.values})
value_df['avg_rev_per_ride'] = value_df['avg_rev_per_ride'].apply(lambda x: x[0])

In [0]:
# verifying whether we calculated average revenue
# per ride for driver 002be0ffdc997bd5c50703158b7c2491
print('AVG Rev per Ride is:')
print(merged[merged['driver_id'] == '002be0ffdc997bd5c50703158b7c2491']['driver_revenue'].unique() / merged[merged['driver_id'] == '002be0ffdc997bd5c50703158b7c2491']['ride_id'].nunique())
print(value_df[value_df['driver_id'] == '002be0ffdc997bd5c50703158b7c2491'])

AVG Rev per Ride is:
[9.41797087]
                          driver_id  avg_rev_per_ride
0  002be0ffdc997bd5c50703158b7c2491          9.417971


We verified that the average revenue per ride is calculated right. 

In [0]:
value_df['avg_rev_per_ride'].describe()

count    837.000000
mean      10.300526
std        1.776365
min        6.449363
25%        9.165920
50%        9.894338
75%       11.087197
max       26.143721
Name: avg_rev_per_ride, dtype: float64

The average revenue per ride is about 10 dollars. 

***What day of week has the most ride?***
We are going to convert all timestamp to day of week. Since each timestamp is associated with an event of a ride, and since each ride has the same number of event (5), then we just need to find the one that appears the most.

In [0]:
merged['dow'] = merged['timestamp'].dt.day_name()
merged['dow'].describe()

count     921044
unique         7
top       Friday
freq      160961
Name: dow, dtype: object

The day with the most rides is Friday.

In [0]:
merged.groupby(['dow'])['fare'].sum()

dow
Friday       1.615512e+06
Monday       1.069051e+06
Saturday     1.505568e+06
Sunday       1.184089e+06
Thursday     1.437297e+06
Tuesday      1.196490e+06
Wednesday    1.309050e+06
Name: fare, dtype: float64

In [0]:
print('Total revenue is: ')
merged['fare'].sum() / 5

Total revenue is: 


1863413.034823614

In [0]:
print(max(ts['timestamp']))
print(min(ts['timestamp']))
print(max(ts['timestamp']) - min(ts['timestamp']))

2016-06-27 00:50:50
2016-03-28 05:48:18
90 days 19:02:32


Friday also makes the most money with 5.8 million dollars. For SF alone. 

So for SF, Lyft makes about 1.86 millions in 90 days in 2016.

For year 2016, Lyft made about 700 millions. 

So SF contributed to 1.077% of Lyft's total revenue in the year of 2016.

(1.86 / 90 * 365) / 700



In [0]:
merged['tod'] = merged['timestamp'].dt.hour
merged.head(10)

,driver_id,driver_onboard_date,ride_id,ride_distance,ride_duration,ride_prime_time,event,timestamp,last_time_drove,life_time,fare,driver_revenue,life_time_revenue,dow,tod
0,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,requested_at,2016-04-23 02:13:50,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793,30.181118,Saturday,2.0
1,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,accepted_at,2016-04-23 02:14:15,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793,30.181118,Saturday,2.0
2,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,arrived_at,2016-04-23 02:16:36,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793,30.181118,Saturday,2.0
3,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,picked_up_at,2016-04-23 02:16:40,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793,30.181118,Saturday,2.0
4,002be0ffdc997bd5c50703158b7c2491,2016-03-29,006d61cf7446e682f7bc50b0f8a5bea5,1811,327,50,dropped_off_at,2016-04-23 02:22:07,2016-06-23 10:29:53,86 days 10:29:53,4.493102,2608.77793,30.181118,Saturday,2.0
5,002be0ffdc997bd5c50703158b7c2491,2016-03-29,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,requested_at,2016-03-29 19:00:49,2016-06-23 10:29:53,86 days 10:29:53,7.368747,2608.77793,30.181118,Tuesday,19.0
6,002be0ffdc997bd5c50703158b7c2491,2016-03-29,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,accepted_at,2016-03-29 19:00:52,2016-06-23 10:29:53,86 days 10:29:53,7.368747,2608.77793,30.181118,Tuesday,19.0
7,002be0ffdc997bd5c50703158b7c2491,2016-03-29,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,arrived_at,2016-03-29 19:03:57,2016-06-23 10:29:53,86 days 10:29:53,7.368747,2608.77793,30.181118,Tuesday,19.0
8,002be0ffdc997bd5c50703158b7c2491,2016-03-29,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,picked_up_at,2016-03-29 19:04:01,2016-06-23 10:29:53,86 days 10:29:53,7.368747,2608.77793,30.181118,Tuesday,19.0
9,002be0ffdc997bd5c50703158b7c2491,2016-03-29,01b522c5c3a756fbdb12e95e87507eda,3362,809,0,dropped_off_at,2016-03-29 19:17:30,2016-06-23 10:29:53,86 days 10:29:53,7.368747,2608.77793,30.181118,Tuesday,19.0


In [0]:
#merged.groupby(['tod']).describe()
#divide it by 5 because of 5 events per each ride_id
merged.groupby(['tod']).count().ride_id / 5

tod
0.0      6558.0
1.0      4784.4
2.0      2961.4
3.0      1447.2
4.0      1175.0
5.0      1584.4
6.0      3222.8
7.0      6278.8
8.0      8380.4
9.0      9293.8
10.0     8262.4
11.0     7841.0
12.0     7750.8
13.0     7431.0
14.0     7522.8
15.0     8094.4
16.0     9114.0
17.0    10867.8
18.0    13106.2
19.0    13419.6
20.0    12195.4
21.0    12190.8
22.0    11421.6
23.0     9304.8
Name: ride_id, dtype: float64

From above observation, the most ride occurs at 6 to 9 PM.

In [0]:
merged.groupby(['tod'])['fare'].sum() / 5

tod
0.0      63979.784915
1.0      48964.690885
2.0      30993.605208
3.0      17419.738818
4.0      17238.849016
5.0      22570.907215
6.0      39218.427445
7.0      68535.060819
8.0      86538.303069
9.0      94338.221707
10.0     87203.814110
11.0     84036.869182
12.0     81898.945708
13.0     76277.721373
14.0     78368.104441
15.0     85594.580718
16.0     93416.326386
17.0    108764.176751
18.0    124308.532819
19.0    123649.708568
20.0    115032.020790
21.0    116013.840974
22.0    110170.347266
23.0     88878.800190
Name: fare, dtype: float64

It also shows that during 6 - 9 PM, Lyft makes the most revenue as well. 

So the time to make the most revenue is about 6-9 PM on a Friday night. 

Next we will do analysis on primtime. 

In [0]:
# compare the revenue with primetime
# to no primetime 
print('Revenue for all rides with primetime')
print(sum(merged[merged['ride_prime_time'] > 0]['fare']) / 5)
print('Revenue for all rides with no primetime')
print(sum(merged[merged['ride_prime_time'] == 0]['fare']) / 5)

Revenue for all rides with primetime
619149.3439871473
Revenue for all rides with no primetime
1244263.6908364398


Revenue for rides with no prime time
is 1.24 millions dollars while rides with some primtime is 619 thousands dollars.

The next questions are how long and far do they drive per day?

In [0]:
# compare the number of rides with primetime
# to no primetime 
print('Number of rides for all rides with primetime')
print(len(merged[merged['ride_prime_time'] > 0]['ride_id']) / 5)
print('Number for all rides with no primetime')
print(len(merged[merged['ride_prime_time'] == 0]['ride_id']) / 5)

Number of rides for all rides with primetime
65102.0
Number for all rides with no primetime
119107.0


In [0]:
# looking at primetime get per day
# in seconds
merged['ttl_prime_time'] = merged.groupby(['driver_id'])['ride_prime_time'].transform(sum) / 5
merged['ttl_prime_time'].describe()

count    921045.000000
mean       6473.896498
std        3772.577760
min           0.000000
25%        4000.000000
50%        5900.000000
75%        8375.000000
max       19675.000000
Name: ttl_prime_time, dtype: float64

We have 4 segments:
0 -> 4000 

4000 -> 5900

5900 -> 8375

8375 > 

For each of these segment, we are going to find the average life time of a driver

In [0]:
time_segments = [4000, 5900, 8375, 19675]
minimum = 0
maximum = 19675
for i in range(len(time_segments)):
  end = time_segments[i]
  if i == 0:
    start = minimum
  if i == len(time_segments) - 1:
    end = maximum
  sum_lt = (merged[merged['ttl_prime_time'].between(start, end)]['life_time'] / pd.to_timedelta(1, unit='D')).unique().sum()
  drivers = (merged[merged['ttl_prime_time'].between(start, end)]['life_time'] / pd.to_timedelta(1, unit='D')).nunique()
  print(sum_lt / drivers)
  start = time_segments[i]
  

48.18071904550828
65.34008398261068
65.52292546296296
66.16876809280545


For those with 0 to 4000 thousand seconds of primetime, the average life time is 48 days.

After 4000 seconds, it's about 65 or 66 days. 

In [0]:
# Next question is does the number of ride a rider get affect
# his or her life time value(i.e do they stay for long)
#

In [0]:
# Lyft for working professional (look at weekday and during working hours like
# 9 AM and 5 or 6 PM)
#